In [27]:
import os
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, 
    confusion_matrix, 
    classification_report
    )

In [28]:
def fit_and_eval(
        X_train: pd.DataFrame,
        X_test: pd.DataFrame,
        y_train: pd.Series,
        y_test: pd.Series,
        feature_names: list = None,
        random_state: int = 42
) -> dict:
    """
    Обучает Logistic Regression и возвращает:
    - пайплайн (по сути pipe подготавливает данные сама как StandardScaler и обучает модель как LogisticRegression)
    - метрики качества 
    - коэфиценты (Показывает, насколько сильно каждый признак влияет на вероятность класса)
    - важность признаков (какие признаки самые значимые для модели)
    """

    #пайплайн: стандартизация + логистическая регрессия
    pipe = make_pipeline(
        StandardScaler(),
        LogisticRegression(max_iter=2000, random_state=random_state)
    )

    #обучение 
    pipe.fit(X_train, y_train)

    #предсказание
    y_pred = pipe.predict(X_test)

    #метрики 
    acc = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    report = classification_report(y_test, y_pred, digits=4) #digits=4 для того, чтобы после десятичного числа показывалось 4 цифры, так как метрики числа с десятичными дробями

    #коэфициенты
    log_reg = pipe.named_steps["logisticregression"] #в этой переменной находится сама ml-модель 
    coefs = log_reg.coef_

    #важность признаков (среднее по классам)
    feat_imp = np.mean(np.abs(coefs), axis=0)
    if feature_names is None:
        feature_names = [f"f{i}" for i in range (len(feat_imp))] #призваивание имен автоматически

    feat_imp_series = pd.Series(feat_imp, index=feature_names).sort_values(ascending=False) #индексы - подписи строк, feat_imp - числовые значения, сортируем по убыванию важности

    return {
        "pipe": pipe,
        "acc": acc,
        "cm": cm,
        "report": report,
        "coef": coefs,
        "feature_imp": feat_imp_series
    }

In [29]:
data_path = os.path.join('..', 'data', 'iris_data.csv')
df = pd.read_csv(data_path)
target_names = ['setosa','versicolor','virginica'] #название видов
df["species"] = df["target"].map({i: name for i, name in enumerate(target_names)})
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target,species
0,5.1,3.5,1.4,0.2,0,setosa
1,4.9,3.0,1.4,0.2,0,setosa
2,4.7,3.2,1.3,0.2,0,setosa
3,4.6,3.1,1.5,0.2,0,setosa
4,5.0,3.6,1.4,0.2,0,setosa


In [30]:
X = df.drop(columns=["target","species"])
y = df["target"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y #все X с большой, чтобы напоминать, что в переменой хранться матрица признаков, обычно двумерная: строки = объекты (наблюдения), столбцы = признаки
)

In [31]:
feature_to_drop = 'sepal width (cm)'
X_train_reduce = X_train.drop(columns=feature_to_drop)
X_test_reduce = X_test.drop(columns=feature_to_drop)

In [32]:
res_full  = fit_and_eval(
    X_train, X_test,
    y_train, y_test,
    feature_names=list(X_test.columns)
)

print("=== Full model (все признаки) ===")
print("Accuracy (test):", res_full["acc"])
print(res_full["report"])

res_reduced = fit_and_eval(
    X_train_reduced, X_test_reduce,
    y_train, y_test,
    feature_names=list(X_test_reduce.columns)
)

print(f"=== Reduced model (без признака {feature_to_drop}) ===")
print("Accuracy (test):", res_reduced["acc"])
print(res_reduced["report"])

delta = res_reduced["acc"] - res_full["acc"]
print(f"\nРазница между Accuracy (reduced - full) = {delta:.4f}")
# Таблица важности признаков
print("=== Важность признаков (full model) ===")
print(res_full["feature_imp"])
#Вывод по эксперементу
print("- Удаление признака 'sepal width (cm)' не изменило точность модели (accuracy = 0.9111).")
print("- Это значит, что данный признак слабо влияет на предсказания, его информация уже дублируется другими признаками.")
print("- Наибольшую значимость для классификации показывают признаки, связанные с лепестками:")
print("  • petal length (cm)")
print("  • petal width (cm)")
print("- Признаки чашелистиков (особенно sepal width) менее информативны.")

=== Full model (все признаки) ===
Accuracy (test): 0.9111111111111111
              precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000        15
           1     0.8235    0.9333    0.8750        15
           2     0.9231    0.8000    0.8571        15

    accuracy                         0.9111        45
   macro avg     0.9155    0.9111    0.9107        45
weighted avg     0.9155    0.9111    0.9107        45

=== Reduced model (без признака sepal width (cm)) ===
Accuracy (test): 0.9111111111111111
              precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000        15
           1     0.8235    0.9333    0.8750        15
           2     0.9231    0.8000    0.8571        15

    accuracy                         0.9111        45
   macro avg     0.9155    0.9111    0.9107        45
weighted avg     0.9155    0.9111    0.9107        45


Разница между Accuracy (reduced - full) = 0.0000
=== Важность признаков (full